In [ ]:
import obi_one as obi
from obi_one.core.info import Info
from obi_one.scientific.from_id.ion_channel_recording_from_id import IonChannelRecordingFromID
from obi_one.scientific.blocks import ion_channel_equations
from obi_one.scientific.tasks import ion_channel_modeling
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST
from entitysdk import Client, ProjectContext

from obi_auth import get_token

obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

# IonChannelRecording IDs

In [ ]:
recording_id = "d6bd64e6-6371-4649-a013-9192923a1e60" # only one id

# Create Ion Channel Fitting Form using Ion Channel Recording(s)

In [ ]:
icm_conf = ion_channel_modeling.IonChannelFittingScanConfig(
    initialize=ion_channel_modeling.IonChannelFittingScanConfig.Initialize(
            recordings=IonChannelRecordingFromID(id_str=recording_id),
            ion_channel_name="Kv1_1",
            # ion="K",
            # temperature=35,
        ),
    info=Info(
        campaign_name="Ion Channel Modeling Campaigne Test 001",
        campaign_description="Just testing",
    ),
    minf_eq=ion_channel_equations.SigFitMInf(),
    mtau_eq=ion_channel_equations.ThermoFitMTauV2(),
    hinf_eq=ion_channel_equations.SigFitHInf(),
    htau_eq=ion_channel_equations.SigFitHTau(),
    gate_exponents=ion_channel_modeling.IonChannelFittingScanConfig.GateExponents(
        m_power=1,
        h_power=1,
    ),
    # stimulus_voltage_exclusion=ion_channel_modeling.IonChannelFittingScanConfig.StimulusVoltageExclusion(), # here the default values are used
    # stimulus_timings=ion_channel_modeling.IonChannelFittingScanConfig.StimulusTimings(),  # here the default values are used
)

# Validated Config
validated_icm_conf = icm_conf.validated_config()

# Generate GridScanGenerationTask (locally) - Note strange issue described for relative paths

In [ ]:
import tempfile

# Create a temporary directory (not used in this example, but can be useful for testing)
with tempfile.TemporaryDirectory() as temp_dir:

    """
    Very strange issue with adding assets when a relative path is used here. Need to look into after CNS.
    """
    # grid_scan = obi.GridScanGenerationTask(form=validated_icm_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../../../obi-output/ion_channel_modeling/grid_scan')
    # grid_scan = obi.GridScanGenerationTask(form=validated_icm_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../../../obi-output/ion_channel_modeling/grid_scan')
    grid_scan = obi.GridScanGenerationTask(form=validated_icm_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    grid_scan.execute(db_client=db_client)
    obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)



# Test endpoint. Service must be launched first (make run-local)

In [ ]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/ion-channel-fitting-scan-config-generate-grid"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json",
}

if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request
request_body = {
    "type": "IonChannelFittingScanConfig",
    "info": {
        "type": "Info",
        "campaign_name": "Ion Channel Modeling Campaigne Test 001",
        "campaign_description": "Just testing",
    },
    "initialize": {
        # "type": "IonChannelFittingScanConfig.Initialize",
        "recordings": 
            {
                "type": "IonChannelRecordingFromID", 
                "id_str": recording_id
            }
        ,
        "ion_channel_name": "Kv1_1",
        # "ion": "k",
        # "temperature": 35,
    },
    "minf_eq": {
        "type": "SigFitMInf"
    },
    "mtau_eq": {
        "type": "ThermoFitMTauV2"
    },
    "hinf_eq": {
        "type": "SigFitHInf"
    },
    "htau_eq": {
        "type": "SigFitHTau"
    },
    # Correct type name per schema: GateExponents
    "gate_exponents": {
        "type": "IonChannelFittingScanConfig.GateExponents",
        "m_power": 1,
        "h_power": 1,
    },
    # Optional advanced fields expected by the API
    # "expert": {"type": "IonChannelFittingScanConfig.Expert"},
    # "brain_region_id": "4642cddb-4fbe-4aae-bbf7-0946d6ada066",
    # "subject_id": "9edb44c6-33b5-403b-8ab6-0890cfb12d07",
    # neuron_block.global must be a list according to the API validation
    # "neuron_block": {"global": []},
}

response = requests.post(url, headers=headers, json=request_body)

if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")